<a href="https://colab.research.google.com/github/puneetpushkar/NIFTY50-Backtest-RSI/blob/main/nifty50_Backtest_RSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Analyse {
Buy NIFTY 50 when RSI crosses above 30 and exit above 70 }

###Libraries Import

In [ ]:
pip install yfinance

In [ ]:
!pip install talib-binary

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
import copy
import math

### Code

In [ ]:
symbol= "^NSEI"
df= yf.Ticker(symbol).history(period= "10y", interval= "1d")

In [ ]:
df["RSI_14"]= talib.RSI(df["Close"], timeperiod= 14)

In [ ]:
symbol_trades= []
trade= {"Symbol":None, "Buy/Sell":None, "Entry":None, "Entry Date": None, "Exit": None, "Exit Date": None}
position= None
m = df.shape[0]
    
for i in range(14, m-1):
  if not math.isnan(df.iloc[i][7]) and df.iloc[i][7] > 30 and df.iloc[i-1][7] <= 30 and position != "Buy":
     
     if trade["Symbol"] is not None:
       trade['Exit'] = df.iloc[i+1][3]
       trade['Exit Date'] = df.iloc[:,[3]].index[i+1]
       symbol_trades.append(copy.deepcopy(trade))


     if position is not None:
        trade["Symbol"]= symbol
        trade["Buy/Sell"]= "Buy"
        trade["Entry"]= (df.iloc[i+1][3])
        trade["Entry Date"]= df.iloc[:,[3]].index[i+1]
     
     position= "Buy"

  if not math.isnan(df.iloc[i][7]) and df.iloc[i][7] > 70 and position != "Sell":
    if trade["Symbol"] is not None:
            trade['Exit']= df.iloc[i+1][3]
            trade['Exit Date'] = df.iloc[:,[3]].index[i+1]
            symbol_trades.append(copy.deepcopy(trade))
        
    if position is not None:
            trade["Symbol"]= symbol
            trade["Buy/Sell"]= "Sell"
            trade["Exit"]= df.iloc[i+1][3]
            trade["Entry Date"]= df.iloc[:,[3]].index[i+1]
    
    position= "Sell"

Tf= pd.DataFrame(symbol_trades)

Tf["P/L"]= np.where(Tf["Buy/Sell"] == "Buy", 100*(Tf["Exit"]-Tf["Entry"])/Tf["Entry"], 100*(Tf["Entry"]-Tf["Exit"])/Tf["Entry"])
Tf['Return']= Tf['P/L'].cumsum()
Tf['Drawdown']= Tf['Return'] - (Tf['Return'].cummax().apply(lambda x:x if x>0 else 0))

In [ ]:
Tf= Tf[Tf["Buy/Sell"] == "Buy"].reset_index(drop= True)

In [ ]:
Tf